In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
train = pd.read_csv("案例_泰坦尼克号存活率预测-基于逻辑回归模型/train.csv")
test = pd.read_csv("案例_泰坦尼克号存活率预测-基于逻辑回归模型/test.csv")

In [ ]:
train = train.drop(labels=["Name","Ticket","Cabin"],axis=1)
test = test.drop(labels=["Name","Ticket","Cabin"],axis=1)

In [ ]:
labels = train["Embarked"].unique().tolist()
train["Embarked"] = train["Embarked"].apply(lambda x: labels.index(x))
labels_test = test["Embarked"].unique().tolist()
test["Embarked"] = test["Embarked"].apply(lambda x: labels_test.index(x))

In [ ]:
train["Sex"] = (train["Sex"] == "male").astype("int")
test["Sex"] = (test["Sex"] == "male").astype("int")

In [ ]:
train["Age"] = train["Age"].fillna(np.mean(train["Age"]))

In [ ]:
test["Age"] = test["Age"].fillna(np.mean(test["Age"]))

In [ ]:
test["Fare"] = test["Fare"].fillna(np.mean(train["Fare"]))

In [ ]:
test.info()

In [ ]:
X = train.iloc[:, train.columns != "Survived"]
y = train.iloc[:, train.columns == "Survived"]

In [ ]:
X_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

In [ ]:
for i in [X_train,x_test,y_train,y_test]:
    i.index = range(i.shape[0])

In [ ]:
clf = DecisionTreeClassifier(random_state=0)
clf = clf.fit(X_train,y_train)
score = clf.score(x_test,y_test)
cross_score = cross_val_score(clf,X,y,cv=10).mean()
cross_score

In [ ]:
tr = []
te = []
for i in range(10):
    clf = DecisionTreeClassifier(random_state=0,
                                 max_depth=i+1,
                                )
    clf = clf.fit(X_train,y_train)
    score_tr = clf.score(X_train,y_train)
    score_te = cross_val_score(clf,X,y,cv=10).mean()
    tr.append(score_tr)
    te.append(score_te)
    
print(max(te))
plt.plot(range(1,11),tr,color="red",label="train")
plt.plot(range(1,11),te,color="blue",label="test")
plt.xticks(range(1,11))
plt.legend()
plt.show()

In [ ]:
# import numpy as np
# gini_thresholds = np.linspace(0,0.5,20)

# parameters = {'splitter':('best','random')
#               ,'criterion':("gini","entropy")
#               ,"max_depth":[*range(1,10)]
#               ,'min_samples_leaf':[*range(1,50,5)]
#               ,'min_impurity_decrease':[*np.linspace(0,0.5,20)]
#              }

# clf = DecisionTreeClassifier(random_state=25)
# GS = GridSearchCV(clf, parameters, cv=10)
# GS.fit(X_train,y_train)

# GS.best_params_

# GS.best_score_

In [ ]:
GS.best_params_

In [ ]:
clf = DecisionTreeClassifier(splitter="random",
                            criterion="entropy",
                            max_depth=6,
                            min_impurity_decrease=0.0,
                            min_samples_leaf=6)
clf = clf.fit(X_train,y_train)
score = clf.score(x_test,y_test)
cross_score = cross_val_score(clf,X,y,cv=10).mean()
cross_score

In [ ]:
test.shape

In [ ]:
predict = clf.predict(test)

In [ ]:
test.info()

In [ ]:
PassengerId = test.PassengerId
submission = pd.DataFrame({"PassengerId": PassengerId, "Survived": predict})

In [ ]:
submission.to_csv("submission.csv",index=False)